## Custom Tutorial 1: Creating a custom model

If you have not done so already, install PyBEAM. Go to the PyBEAM github (https://github.com/MatthewMurrow/pybeam) for instructions.

Once PyBEAM is installed, we can create our first model. First, download the folder "custom_model_template.zip" and unzip it. Then, copy the files to a new directory.

To do this, create a new folder anywhere on your device and copy the files located in folder "custom_model_template" to it. There will be three files you need to copy: "setup.py", "functions.pyx", and "model.pyx". The only file that needs to be modified is "model.pyx". DO NOT TOUCH THE OTHER TWO FILES.

Un-edited, "model.pyx" will have the following appearance:


The first two lines,

    import cython
    cimport cython
    
imports Cython. The next line,

    from libc.math cimport pow, exp, log, sin, cos
    
imports useful C math functions. The first, pow, is the C verison of raising a number to a power. It's syntax is pow(a, b), which is the python equivalent of a**b. Whenever using exponents, always use pow because it is substantially faster when compiled than the python equivalent.

The next several functions (exp, log, sin, and cos) all act the same as their python equivalents. These functions are imported since they are commonly used in models in the literature, but other functions can be included by adding them to this list. For example, if you want a tangent function, you would add tan to the list, giving,

    from libc.math cimport pow, exp, log, sin, cos, tan
    
The next line,

    from libc.stdlib cimport srand, rand, RAND_MAX
    
imports other useful C functions that might be needed in a custom model. rand is C's random number generator, while srand seeds the random number generator. RAND_MAX is the max value of the random number generator, allowing the generator to produce any number (see C documentation for further discussion of these functions if you are in need of them).

Note that, if functions from other C libraries are desired, additional import lines can be added here.

The second block of code is where your model is defined. First, you must set "N_phi" equal to the number of parameters your model contains. If, for example, you were defining the base model described in Default Tutorial 1, you would input N_phi = 4 here since there were 4 parameters: t_nd, w, mu, and a.

Next we define outputs for our model functions. There are eight model related functions that can be modified to fit your needs (note that ONLY the outputs, NOT the inputs, can be changed). 

The first, non_decision_time, sets the model's non-decision time. It accepts an input of phi, the list of all the parameters in your model (something we will use later in the tutorial). Since this function only accepts phi as an input, it can only output a constant value.

The second, relative start, sets the model's relative start point. Like non_decision_time, it only accepts an input of phi, meaning it outputs a constant value only.

The next two functions, drift and diffusion, set the model's drift (v(x,t) in PyBEAM's publication) and diffusion (D(x,t) in PyBEAM's publication) rates. They both accept three inputs: phi, x, and t. phi, as before, contains the parameters used by your model. The second, x, is the spatial coordinate and corresponds to the accumulator state. The last, t, is time. Thus, both the drift and diffusion rates can have time and spatial dependecies.

The next two functions, upper_decision_threshold and lower_decision_threshold, set the locations of the decision thresholds. These two functions accepts inputs of phi and t, meaning that the thresholds can be time dependent.

The last two model functions are for adding a contamination model. The first of these, contamination_strength, sets the amount of contamination. If it returns 0, your model has no contamination; if it returns 1, your model is only contamination. This functions only accepts inputs of phi, meaning it can only output a constant value.

The second contamination function is contamination_probability. This functions returns the probability of a contamination event occurring at time t. For example, in Tutorial 5, a uniform contamination was added to the model. So, a uniform distribution is required for this function. It accepts inputs of phi and t.

The last block of code provides a function for modifying the time step. In general this should not be touched, but it allows for your time step to be made altered in regions where it's hard to accurately obtain the likelihood function. If it outputs 1, the time step is not modified; if it recieves a value of less than 1, the time step will get smaller; if it outputs a value more than 1; the time step will grow. For example, if it returns 0.5, the time step will be one-half of its normal value.


As an example, lets define a model we have already seem from earlier PyBEAM tutorials. It will start with the base model from PyBEAM's default sub-module, and add an exponential threshold and uniform contamination. Doing so produces the following model.pyx file:


Once you have built your file, you need to compile it. To do so, open Command Prompt, Temrinal, or Anaconda prompt (depending on your OS) and navigate change directory to where you have saved your model.pyx file. Then, run the following line of code:

    python setup.py build_ext --inplace
    
You have now built your first model! Move on to the next tutorial to learn how to use the model.
